In [13]:
import requests
import json
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec
import glob
import os
import matplotlib.image as mpimg
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io

#URL = 'http://143.248.94.189:35005/api/predict'
#URL = 'http://143.248.96.81:35005/api/predict'
URL = 'http://143.248.96.69:35005/api/predictIndex'
#URLGetColor = 'http://143.248.96.69:35005/api/colorMap'

def convertTuple(list): 
    return tuple(i for i in list) 

def create_ade20k_label_colormap():
      return np.asarray([
      [0, 0, 0],
      [120, 120, 120],
      [180, 120, 120],
      [6, 230, 230],
      [80, 50, 50],
      [4, 200, 3],
      [120, 120, 80],
      [140, 140, 140],
      [204, 5, 255],
      [230, 230, 230],
      [4, 250, 7],
      [224, 5, 255],
      [235, 255, 7],
      [150, 5, 61],
      [120, 120, 70],
      [8, 255, 51],
      [255, 6, 82],
      [143, 255, 140],
      [204, 255, 4],
      [255, 51, 7],
      [204, 70, 3],
      [0, 102, 200],
      [61, 230, 250],
      [255, 6, 51],
      [11, 102, 255],
      [255, 7, 71],
      [255, 9, 224],
      [9, 7, 230],
      [220, 220, 220],
      [255, 9, 92],
      [112, 9, 255],
      [8, 255, 214],
      [7, 255, 224],
      [255, 184, 6],
      [10, 255, 71],
      [255, 41, 10],
      [7, 255, 255],
      [224, 255, 8],
      [102, 8, 255],
      [255, 61, 6],
      [255, 194, 7],
      [255, 122, 8],
      [0, 255, 20],
      [255, 8, 41],
      [255, 5, 153],
      [6, 51, 255],
      [235, 12, 255],
      [160, 150, 20],
      [0, 163, 255],
      [140, 140, 140],
      [250, 10, 15],
      [20, 255, 0],
      [31, 255, 0],
      [255, 31, 0],
      [255, 224, 0],
      [153, 255, 0],
      [0, 0, 255],
      [255, 71, 0],
      [0, 235, 255],
      [0, 173, 255],
      [31, 0, 255],
      [11, 200, 200],
      [255, 82, 0],
      [0, 255, 245],
      [0, 61, 255],
      [0, 255, 112],
      [0, 255, 133],
      [255, 0, 0],
      [255, 163, 0],
      [255, 102, 0],
      [194, 255, 0],
      [0, 143, 255],
      [51, 255, 0],
      [0, 82, 255],
      [0, 255, 41],
      [0, 255, 173],
      [10, 0, 255],
      [173, 255, 0],
      [0, 255, 153],
      [255, 92, 0],
      [255, 0, 255],
      [255, 0, 245],
      [255, 0, 102],
      [255, 173, 0],
      [255, 0, 20],
      [255, 184, 184],
      [0, 31, 255],
      [0, 255, 61],
      [0, 71, 255],
      [255, 0, 204],
      [0, 255, 194],
      [0, 255, 82],
      [0, 10, 255],
      [0, 112, 255],
      [51, 0, 255],
      [0, 194, 255],
      [0, 122, 255],
      [0, 255, 163],
      [255, 153, 0],
      [0, 255, 10],
      [255, 112, 0],
      [143, 255, 0],
      [82, 0, 255],
      [163, 255, 0],
      [255, 235, 0],
      [8, 184, 170],
      [133, 0, 255],
      [0, 255, 92],
      [184, 0, 255],
      [255, 0, 31],
      [0, 184, 255],
      [0, 214, 255],
      [255, 0, 112],
      [92, 255, 0],
      [0, 224, 255],
      [112, 224, 255],
      [70, 184, 160],
      [163, 0, 255],
      [153, 0, 255],
      [71, 255, 0],
      [255, 0, 163],
      [255, 204, 0],
      [255, 0, 143],
      [0, 255, 235],
      [133, 255, 0],
      [255, 0, 235],
      [245, 0, 255],
      [255, 0, 122],
      [255, 245, 0],
      [10, 190, 212],
      [214, 255, 0],
      [0, 204, 255],
      [20, 0, 255],
      [255, 255, 0],
      [0, 153, 255],
      [0, 41, 255],
      [0, 255, 204],
      [41, 0, 255],
      [41, 255, 0],
      [173, 0, 255],
      [0, 245, 255],
      [71, 0, 255],
      [122, 0, 255],
      [0, 255, 184],
      [0, 92, 255],
      [184, 255, 0],
      [0, 133, 255],
      [255, 214, 0],
      [25, 194, 194],
      [102, 255, 0],
      [92, 0, 255],
  ], dtype=np.uint8)

def label_to_color_image(label):
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create_ade20k_label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')
    return colormap[label]

def vis_segmentation(image, seg_map, index):
    """Visualizes input image, segmentation map and overlay view."""
    plt.figure(figsize=(15, 5))
    grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

    plt.subplot(grid_spec[0])
    plt.imshow(image)
    plt.axis('off')
    plt.title('input image')

    plt.subplot(grid_spec[1])
    seg_image = label_to_color_image(seg_map).astype(np.uint8)
    im2 = Image.fromarray(seg_image, 'RGB')
    seg_image = im2.resize(image.size, Image.BILINEAR)

    plt.imshow(seg_image)
    plt.axis('off')
    plt.title('segmentation map')

    plt.subplot(grid_spec[2])
    plt.imshow(image)
    plt.imshow(seg_image, alpha=0.7)
    plt.axis('off')
    plt.title('segmentation overlay')

    unique_labels = np.unique(seg_map)
    #print(FULL_COLOR_MAP[unique_labels])
    ax = plt.subplot(grid_spec[3])
    plt.imshow(
        FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
    ax.yaxis.tick_right()
    plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
    plt.xticks([], [])
    ax.tick_params(width=0.0)
    plt.grid('off')
    plt.show()
   
def intersectionAndUnion(imPred, imLab, numClass):
    imPred = np.asarray(imPred)
    imLab = np.asarray(imLab)

    # Remove classes from unlabeled pixels in gt image. 
    # We should not penalize detections in unlabeled portions of the image.
    imPred = imPred * (imLab>0)

    # Compute area intersection:
    intersection = imPred * (imPred==imLab)
    (area_intersection,_) = np.histogram(intersection, bins=numClass, range=(1, numClass))

    # Compute area union:
    (area_pred,_) = np.histogram(imPred, bins=numClass, range=(1, numClass))
    (area_lab,_) = np.histogram(imLab, bins=numClass, range=(1, numClass))
    area_union = area_pred + area_lab - area_intersection
    
    return (area_intersection, area_union)

# This function takes the prediction and label of a single image, returns pixel-wise accuracy
# To compute over many images do:
# for i = range(Nimages):
#   (pixel_accuracy[i], pixel_correct[i], pixel_labeled[i]) = pixelAccuracy(imPred[i], imLab[i])
# mean_pixel_accuracy = 1.0 * np.sum(pixel_correct) / (np.spacing(1) + np.sum(pixel_labeled))
def pixelAccuracy(imPred, imLab):
    imPred = np.asarray(imPred)
    imLab = np.asarray(imLab)

    # Remove classes from unlabeled pixels in gt image. 
    # We should not penalize detections in unlabeled portions of the image.
    pixel_labeled = np.sum(imLab>0)
    pixel_correct = np.sum((imPred==imLab)*(imLab>0))
    pixel_accuracy = 1.0 * pixel_correct / pixel_labeled

if __name__ == "__main__":
    LABEL_NAMES = np.array(['wall' ,'building' ,'sky' ,'floor' ,'tree' ,'ceiling' ,'road' ,'bed' ,'windowpane' ,'grass' ,'cabinet' ,'sidewalk' ,'person' ,'earth' ,'door' ,'table' ,'mountain' ,'plant' ,'curtain' ,'chair' ,'car' ,'water' ,'painting' ,'sofa' ,'shelf' ,'house' ,'sea' ,'mirror' ,'rug' ,'field' ,'armchair' ,'seat' ,'fence' ,'desk' ,'rock' ,'wardrobe' ,'lamp' ,'bathtub' ,'railing' ,'cushion' ,'base' ,'box' ,'column' ,'signboard' ,'chest of drawers' ,'counter' ,'sand' ,'sink' ,'skyscraper' ,'fireplace' ,'refrigerator' ,'grandstand' ,'path' ,'stairs' ,'runway' ,'case' ,'pool table' ,'pillow' ,'screen door' ,'stairway' ,'river' ,'bridge' ,'bookcase' ,'blind' ,'coffee table' ,'toilet' ,'flower' ,'book' ,'hill' ,'bench' ,'countertop' ,'stove' ,'palm' ,'kitchen island' ,'computer' ,'swivel chair' ,'boat' ,'bar' ,'arcade machine' ,'hovel' ,'bus' ,'towel' ,'light' ,'truck' ,'tower' ,'chandelier' ,'awning' ,'streetlight' ,'booth' ,'television' ,'airplane' ,'dirt track' ,'apparel' ,'pole' ,'land' ,'bannister' ,'escalator' ,'ottoman' ,'bottle' ,'buffet' ,'poster' ,'stage' ,'van' ,'ship' ,'fountain' ,'conveyer belt' ,'canopy' ,'washer' ,'plaything' ,'swimming pool' ,'stool' ,'barrel' ,'basket' ,'waterfall' ,'tent' ,'bag' ,'minibike' ,'cradle' ,'oven' ,'ball' ,'food' ,'step' ,'tank' ,'trade name' ,'microwave' ,'pot' ,'animal' ,'bicycle' ,'lake' ,'dishwasher' ,'screen' ,'blanket' ,'sculpture' ,'hood' ,'sconce' ,'vase' ,'traffic light' ,'tray' ,'ashcan' ,'fan' ,'pier' ,'crt screen' ,'plate' ,'monitor' ,'bulletin board' ,'shower' ,'radiator' ,'glass' ,'clock' ,'flag'])
    FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
    FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)
    
    listIOU = []
    
    listRGBCV = []
    listRGBImage = []
    listGT = []
    for file in glob.glob('../../Dataset/ade20k_images/*.jpg'):
        img_import = cv2.imread(file)
        listRGBCV.append(img_import)
        img_import2 = Image.open(file)
        listRGBImage.append(img_import2)
        filenameExt = os.path.basename(file)
        filename,_ = os.path.splitext(filenameExt)
        img_gt_path = "../../Dataset/Converted/" + filename + "_seg.png"
        img_gt = Image.open(img_gt_path)
        img_gt = img_gt.convert("RGB")
        listGT.append(img_gt)
        
        
    
    #img_path = '../../Dataset/ade20k_images/ADE_val_00000506.jpg'
    #im1 = cv2.imread(img_path)
    #img_gt_path = '../../Dataset/Converted/ADE_val_00000506_seg.png'
    #img_gt = Image.open(img_gt_path)
    #img_gt = img_gt.convert("RGB")
    
    print("working...")
    outtxt = open("result.txt","w") 
    for index in range(len(listRGBCV)):
        img = listRGBCV[index]
        img_gt = listGT[index]
        print(index)
        
        rNum, cNum = img.shape[:2]
        im1=cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        strImgTest = '['
        for r in range(rNum):
            strImgTest+='['
            for c in range(cNum):
                strTmp = '['+str((im1[r][c][0]))+','+str((im1[r][c][1]))+','+str((im1[r][c][2]))+']'
                strImgTest+=strTmp

                if c !=cNum-1 :
                    strImgTest+=','
            strImgTest+=']'
            if r != rNum-1:
                strImgTest += ','
        strImgTest+=']'

        res = requests.post(url=URL, data={'image':strImgTest})
        #print(res.text)
        param = json.loads(res.text)
        #with open('jsonResult.txt', 'w') as json_file:
        #    json.dump(param, json_file)
        #print("received result")
        
        seg_res = param['seg_index']
        seg_map = np.array(seg_res, dtype=np.uint8)
        
        for row in range(len(seg_map)):
            for col in range(len(seg_map[0])):
                seg_map[row, col] -= 1
                x = seg_map[row, col]
                #if(LABEL_NAMES[x] != 'door' and LABEL_NAMES[x] != 'table' and LABEL_NAMES[x] != 'shelf' and LABEL_NAMES[x] != 'sofa'):
                #    seg_map[row, col] = 0
                if(LABEL_NAMES[x] == 'floor'):
                    seg_map[row, col] = 3
                elif(LABEL_NAMES[x] == 'table' or LABEL_NAMES[x] =='desk' or LABEL_NAMES[x] == 'coffee table'):
                    seg_map[row, col] = 15
                elif(LABEL_NAMES[x] == 'sofa' or LABEL_NAMES[x] == 'armchair' or LABEL_NAMES[x] == 'seat' or LABEL_NAMES[x] == 'ottoman' or LABEL_NAMES[x] == 'chair' or LABEL_NAMES[x] == 'stool'):
                    seg_map[row, col] = 23
                elif(LABEL_NAMES[x] == 'shelf' or LABEL_NAMES[x] == 'cabinet' or LABEL_NAMES[x] == 'wardrobe' or LABEL_NAMES[x] == 'chest of drawers' or LABEL_NAMES[x] == 'bookcase'):
                    seg_map[row, col] = 24
                else:
                    seg_map[row, col] = 0
                    
        seg_image = label_to_color_image(seg_map).astype(np.uint8)
        im2 = Image.fromarray(seg_image, 'RGB')
        seg_image = im2.resize(img_gt.size, Image.BILINEAR)

        #im2 =  Image.fromarray(seg_image, 'RGB')
        #im2 = im2.resize((rNum, cNum), Image.BILINEAR)
        #vis_segmentation(listRGBImage[index], seg_map, index)
        
        orgImage = im1
        sw, sh = seg_image.size
        resizedImage = cv2.resize(orgImage, (sw, sh), interpolation = cv2.INTER_AREA)
        #seg_cv_image = np.array(seg_image) 
        #seg_cv_image = seg_cv_image[:, :, ::-1].copy() 
        
        segments = slic(resizedImage, n_segments = 300, compactness = 10, sigma = 2)
        for (i, segVal) in enumerate(np.unique(segments)):
            mask = np.zeros((sh, sw), dtype = "uint8")
            mask[segments == segVal] = 255
            #superSeg = cv2.bitwise_and(resizedImage,resizedImage,mask = mask)
            #segImgConverted = cv2.cvtColor(superSeg,cv2.COLOR_BGR2RGB)
            #superSeg = Image.fromarray(segImgConverted, 'RGB')
            
            dictionary = dict() 
            for x in range(sw):
                for y in range(sh):
                    if mask[y][x] == 255:
                        color = seg_image.getpixel((x, y))
                        if color in dictionary:
                            dictionary[color] += 1
                        else:
                            dictionary[color] = 1
                            
            maxInt = 0
            maxColor = (0, 0, 0)
            for key in dictionary:
                if dictionary[key] > maxInt:
                    maxInt = dictionary[key]
                    maxColor = key
            
            for x in range(sw):
                for y in range(sh):
                    if mask[y][x] == 255:
                        seg_image.putpixel((x, y), maxColor)
                    
            #segFile = "../../Dataset/Superpixels/" + str(index) + " - " + str(i) + ".png"
            #save = superSeg.save(segFile)
        
        #orgImage = listRGBImage[index]
        resizedOrgImage = listRGBImage[index].resize(seg_image.size, Image.BILINEAR)
        convertOrgImage = Image.new('RGBA', size=seg_image.size, color=(0, 0, 0, 0))
        convertOrgImage.paste(resizedOrgImage)
        convertSegImage = Image.new('RGBA', size=seg_image.size, color=(0, 0, 0, 0))
        convertSegImage.paste(seg_image)
        result = Image.blend(convertOrgImage, convertSegImage, alpha=0.7)
        outputFile = "../../Dataset/Seg_result/" + str(index) + ".png"
        save = result.save(outputFile)
        
        # convert rgb seg images to greyscale
        width, height = seg_image.size
        seg_image_gray = np.zeros((height,width), np.uint8)

        for row in range(width):
            for col in range(height):            
                if(seg_image.getpixel((row, col)) == (0, 0, 0)):
                    seg_image_gray[col][row] = 0;
                elif(seg_image.getpixel((row, col)) == (6, 230, 230)):
                    seg_image_gray[col][row] = 4;
                elif(seg_image.getpixel((row, col)) == (8, 255, 51)):
                    seg_image_gray[col][row] = 16;
                elif(seg_image.getpixel((row, col)) == (255, 6, 51)):
                    seg_image_gray[col][row] = 24;
                elif(seg_image.getpixel((row, col)) == (11, 102, 255)):
                    seg_image_gray[col][row] = 25;

        #print(seg_image_gray)
        gt_w, gt_h = img_gt.size
        img_gt_gray = np.zeros((gt_h,gt_w), np.uint8)
        for row in range(gt_w):
            for col in range(gt_h):
                if(img_gt.getpixel((row, col)) == (0, 0, 0)):
                    img_gt_gray[col][row] = 0;
                elif(seg_image.getpixel((row, col)) == (6, 230, 230)):
                    img_gt_gray[col][row] = 4;
                elif(img_gt.getpixel((row, col)) == (8, 255, 51)):
                    img_gt_gray[col][row] = 16;
                elif(img_gt.getpixel((row, col)) == (255, 6, 51)):
                    img_gt_gray[col][row] = 24;
                elif(img_gt.getpixel((row, col)) == (11, 102, 255)):
                    img_gt_gray[col][row] = 25;
                
        
        (area_intersection, area_union) = intersectionAndUnion(seg_image_gray, img_gt_gray, 50)
        IoU = 1.0 * np.sum(area_intersection) / np.sum(np.spacing(1)+area_union)
        stringOutput = "IoU: " + str(IoU)
        print(stringOutput)
        #print(stringOutput)
        listIOU.append(IoU)
        outtxt.write(str(IoU) + " ")
        
        #plt.figure(figsize=(20, 15))
        #plt.imshow(seg_image)
        
    averageIOU = sum(listIOU)/len(listIOU)
    output = "avg: " + str(averageIOU)
    print(output)
    outtxt.close()



working...
0
IoU: 0.9537944882167139
1
IoU: 0.8936816618601854
2
IoU: 0.85255330781848
3
IoU: 0.5764323917630317
4
IoU: 0.7112922225308139
5
IoU: 0.9638287736491085
6
IoU: 0.8980967256739558
7
IoU: 0.9571069557439196
8
IoU: 0.8738753797183181
9
IoU: 0.8366388410063721
10
IoU: 0.9213013124299267
11
IoU: 0.8850328811082402
12
IoU: 0.9650602799056207
13
IoU: 0.8952271508465487
14
IoU: 0.7185698328048429
15
IoU: 0.9449115808318238
16
IoU: 0.9390257749517429
17
IoU: 0.7525199120598237
18
IoU: 0.9015336437136087
19
IoU: 0.9301224337206767
20
IoU: 0.8779013489139745
21
IoU: 0.6292650507229112
22
IoU: 0.8000675740065322
23
IoU: 0.8266226929935758
24
IoU: 0.8955576348972953
25
IoU: 0.8778937563354429
26
IoU: 0.7469370456938844
27
IoU: 0.8471966579116084
28
IoU: 0.8812653609995461
29
IoU: 0.7114534504824944
30
IoU: 0.7912390253063524
31
IoU: 0.7775923111388335
32
IoU: 0.9583052614183838
33
IoU: 0.9863198584812947
34
IoU: 0.9520771855587283
35
IoU: 0.8896827064509351
36
IoU: 0.9467535231087101
37